In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
import os
os.chdir("C:\\Users\\SUJANPC\\Documents\\Energies")
%matplotlib inline
import warnings
warnings.simplefilter("ignore")
import numpy
from sklearn.neural_network import *
from pandas import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import *
from sklearn.tree import *
from sklearn.preprocessing import *
import tensorflow.keras.backend as K
import numpy as np
from tensorflow.keras.models import *
from scipy.ndimage import gaussian_filter1d
import seaborn as sns
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tqdm import tqdm
from sklearn.metrics import *
import TSMETRICS
from tensorflow.keras.layers import *
from tensorflow.keras.layers import *
import time
import optuna
from optuna.study import *
from optuna.trial import *
import math
import datetime
import PIALL
import tensorflow as tf

from tensorflow.keras import initializers 


In [ ]:
df= pd.read_excel('1D.xlsx',sheet_name='1D')
data = df.filter(["Lawnton"])
df1=data.dropna()


In [ ]:
df1

In [ ]:
df2=df1
lag =range(1,12)

for col in df2.columns:
    for l in lag:
        df2.loc[:,col+"_"+str(l)] = df2[col].shift(l)

In [ ]:
df2

In [ ]:
df_A= pd.read_excel('1D.xlsx',sheet_name='SILO_Lawnton')
df_A

In [ ]:
df_B=df_A.drop(['YYYY-MM-DD'], axis=1)
df_B

In [ ]:
df_C=pd.concat([df2, df_B], axis=1, join='inner')
df_C

In [ ]:
df5=df_C.dropna()
df5

In [ ]:
df6=df5
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() 
column_names_to_not_normalize = ['Lawnton']
column_names_to_normalize = [x for x in list(df6) if x not in column_names_to_not_normalize ]
x = df6[column_names_to_normalize].values
x_scaled = scaler.fit_transform(x)
df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = df6.index)
df6[column_names_to_normalize] = df_temp

In [ ]:
df6

In [ ]:
Train=df6.iloc[0:3270,0:29]
Test=df6.iloc[3270:,0:29]

In [ ]:
array=Train.values
X1=array[:,1:29]
y1=array[:,0]
time_steps= 1
X_train= np.zeros((X1.shape[0] - time_steps +1, 1, X1.shape[1]))
y_train= np.zeros((y1.shape[0] -time_steps +1,))
for ix in range(X_train.shape[0]):
    for jx in range(time_steps):
        X_train[ix, jx, :]= X1[ix +jx, :]
    y_train[ix]= y1[ix + time_steps -1]
print (X_train.shape, y_train.shape)
array3=Test.values
X1=array3[:,1:29]
y1=array3[:,0]
time_steps= 1
X_test= np.zeros((X1.shape[0] - time_steps +1, 1, X1.shape[1]))
y_test= np.zeros((y1.shape[0] -time_steps +1,))
for ix in range(X_test.shape[0]):
    for jx in range(time_steps):
        X_test[ix, jx, :]= X1[ix +jx, :]
    y_test[ix]= y1[ix + time_steps -1]
print (X_test.shape, y_test.shape)


In [ ]:
import time
import optuna
from optuna.study import *
from optuna.trial import *
import math
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
import datetime
path_best_model = 'CNNBLSTM_OPTUNA_Lawnton.h5'
best_score=0

X_train_shape = X_train.shape
TimeSteps=X_train_shape[1]
Dims=X_train_shape[2]

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

def model_learning_curve(history):
    """
    This function plots a training and testing loss
    """
    
    # Loss with Epoch
    plt.figure('16,6')
    plt.title('Model Loss')
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()


def callbacks():
    DNN_lr_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                         patience=8, verbose=2, factor=0.7,min_delta = 1e-04, cooldown = 0,min_lr =0)
    callbacks = [EarlyStopping(monitor='val_loss', patience=20),
                 ModelCheckpoint(filepath='best_weights_CELM_Kalbar_OPT_SILO.h5',verbose=2,
                                 monitor='val_loss', save_best_only=True),DNN_lr_reduction]
    return callbacks
callbacks_list = callbacks()   

In [ ]:
def objective(trial):
        
    inputs = Input(shape=(TimeSteps, Dims))
    x=Conv1D(
            filters=trial.suggest_int('filters1', 10, 1000, 10),
            kernel_size=trial.suggest_int('kernel_size',2,20,step=2),
            activation=trial.suggest_categorical('activation1', ['relu', 'tanh']),
            padding='same',kernel_initializer="glorot_uniform")(inputs)
    x1=Conv1D(
            filters=trial.suggest_int('filters2', 10, 800, 10),
            kernel_size=1,
            activation=trial.suggest_categorical('activation2', ['relu', 'tanh']),
            padding='same',kernel_initializer="glorot_uniform")(x)
    x2=Conv1D(
            filters=trial.suggest_int('filters3', 10, 800, 10),
            kernel_size=1,
            activation=trial.suggest_categorical('activation3', ['relu', 'tanh']),
            padding='same',kernel_initializer="glorot_uniform")(x1)

    Flatten_=Flatten(data_format=None,name='FLATTEN')(x2)
    x3= Dense(10, activation='sigmoid')(Flatten_ )
    output = Dense(1, activation='sigmoid')(x3 )
    model = Model(inputs=[inputs], outputs=output)   
    optimizer_name = trial.suggest_categorical('optimizer_name', ['Adam', 'RMSprop'])

    if optimizer_name == 'Adam':
        optimizer = Adam(learning_rate=trial.suggest_float('learning_rate',  1e-6, 1e-2, log=True))
    else:
        optimizer = RMSprop(
            learning_rate=trial.suggest_float('learning_rate',  1e-6, 1e-2, log=True),
            momentum=trial.suggest_float('momentum',  0.1, 0.9, log=True),
        )    
        
    # We compile our model with a sampled learning rate.
    model.compile(loss=root_mean_squared_error, optimizer=optimizer,metrics='mse')
    history=model.fit(
        X_train,
        y_train,
        shuffle=False,
        batch_size=trial.suggest_int('batch_size', 1, 20, 1),
        epochs=500,
        validation_split=0.2,
        verbose=2,
        callbacks=callbacks_list
    )

    # Evaluate the model accuracy on the test set.
    YP = model.predict(X_test)
    YT=y_test
    score=root_mean_squared_error(YT.flatten(),YP.flatten())
    model_learning_curve(history)
    return score

In [ ]:
study_name = "cnn_study_Kalbar_Opt_SILO"  # unique identifier of the study.
study = optuna.create_study(
    direction='minimize',
    study_name=study_name,
    load_if_exists=True,
)

study.optimize(objective, timeout=8*60*60) # Timeout in seconds e.g. timeout=20*60*60 

In [ ]:
DNN_lr_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                     patience=8, verbose=2, factor=0.7,min_delta = 1e-04, cooldown = 0,min_lr =0)
callbacks = [EarlyStopping(monitor='val_loss', patience=50),
             ModelCheckpoint(filepath='best_weights_CELM_Lawnton_SILO.h5',verbose=2,
                             monitor='val_loss', save_best_only=True),DNN_lr_reduction]

In [ ]:
#Define CNN Model

def generate_cnn():   
    model = Sequential()
    model.add(Conv1D(filters=20,input_shape=(X_train.shape[1],  X_train.shape[2]),kernel_size=10,activation='relu',
                     padding='same',kernel_initializer="glorot_uniform"))
    model.add(MaxPool1D(pool_size=1))
    model.add(Conv1D(filters=18,kernel_size=8,activation='relu',
                     padding='same',kernel_initializer="glorot_uniform"))
    model.add(MaxPool1D(pool_size=1))
    model.add(Conv1D(filters=14,kernel_size=6,activation='relu',
                     padding='same',kernel_initializer="glorot_uniform"))
    model.add(MaxPool1D(pool_size=1))
    model.add(Flatten(data_format=None,name='FLATTEN'))
    model.add(Dense(units=10,activation='relu')),
    model.add(Dense(units=1,activation='relu')),
    optimizer= RMSprop(learning_rate=0.001, momentum=0.009)
    print(model.summary())
    model.compile(loss='mse', optimizer=optimizer, metrics='mse')
    model.fit(X_train, y_train, epochs=1000,validation_split=0.2, batch_size=2, verbose=2,callbacks=callbacks)      
    return model


In [ ]:
cnn = generate_cnn()  

In [ ]:
from tensorflow.keras.models import load_model
cnn =load_model('best_weights_CELM_Lawnton_SILO.h5')
cnn.summary()

In [ ]:
#Define Hidden Layer
#The Hidden Layer is the layer between CNN and ELM
layer_name = 'FLATTEN'
hidden_layer_model = tf.keras.Model(inputs=cnn.input, outputs=cnn.get_layer(layer_name).output)
Train= hidden_layer_model.predict(X_train)
Test= hidden_layer_model.predict(X_test)

In [ ]:
Train.shape,Test.shape

In [ ]:
from ELM_REG import ELM

In [ ]:
def get_ELM(k, hidden, X_train, y_train, num_classes=1):
    input_length = k
    num_hidden_layers = hidden

    model = ELM(input_length,
                num_hidden_layers,
                num_classes)
    
    model.fit(X_train, y_train, display_time=False)
    return model

In [ ]:
model = get_ELM(14, 10, Train, y_train )

In [ ]:
prediction = model(Test)

In [ ]:
prediction.shape

In [ ]:
YP=prediction
YT=y_test

In [ ]:
import PIALL
PIALL_Test_ESN=PIALL.PI(YT.flatten(),YP.flatten())
PIALL_Test_ESN

In [ ]:
from sklearn.svm import SVC

from hyperopt import fmin, tpe, hp, SparkTrials, STATUS_OK, Trials
from math import sqrt


In [ ]:
def Obj(params):
    model = ELM(**params,num_input_nodes=14,num_out_units=1)
    model.fit(Train, y_train, display_time=False)
    y_pred = model(Test)
    score = sqrt(mean_squared_error(y_test, y_pred))
    print(score)
    return {'loss': score, 'status': STATUS_OK,'Trained_Model': model} 

In [ ]:
# kernel='rbf', degree=3, gamma='scale', coef0=0.0, tol=0.001, C=1.0, 
# epsilon=0.1, shrinking=True, cache_size=200, verbose=False, max_iter=-1
space ={

        'num_hidden_units': hp.choice('num_hidden_units',np.arange(1,1000 , 5, dtype=int)),
        'activation':hp.choice('num_input_nodes', ['sigmoid','hardlimit','fourier'])
    }

In [ ]:
def optimize(trials, space):
    
    best = fmin(Obj, space, algo=tpe.suggest, max_evals=500)
    return best

trials = Trials()


best_params = optimize(trials, space)


In [ ]:
from hyperopt import hp, fmin, tpe, space_eval,Trials
# Return the best parameters
best_hp=space_eval(space, best_params)
best_hp

In [ ]:
model2 = ELM(num_input_nodes=14, num_hidden_units=431, num_out_units=1, activation='sigmoid',
                 loss='mse', beta_init=None, w_init=None, bias_init=None)

In [ ]:
model2.fit(Train, y_train, display_time=False)

In [ ]:
prediction = model2(Test)

In [ ]:
YP=prediction
YT=y_test

In [ ]:
import PIALL
PIALL_Test_CELM=PIALL.PI(YT.flatten(),YP.flatten())
PIALL_Test_CELM

In [ ]:
os.chdir("C:\\Users\\SUJANPC\\Documents\\Energies\\EP2")
TEST=pd.DataFrame([YP.flatten(),YT.flatten()]).T
TEST.columns=['CELM','YTEST']
writer = pd.ExcelWriter('EP2_Lawnton_CELM.xlsx', engine='xlsxwriter')
TEST.to_excel(writer, sheet_name='TEST_Result')
PIALL_Test_CELM.to_excel(writer, sheet_name='PI')
writer.save()